[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shang-vikas/series1-coding-exercises/blob/main/exercises/blog-09/exercise-04.ipynb)

# Fine-tuning GPT-2 on Poetry Dataset

## Goal
Fine-tune GPT-2 on ~10M+ poetry tokens in Colab with:
- Strong stylistic consistency
- No overfitting
- Stable training
- Clean reproducibility

## Assumptions
- Colab with GPU (T4 / A100)
- Gutenberg Poetry Corpus (`gutenberg-poetry.ndjson.gz`) downloaded from [aparrish/gutenberg-poetry-corpus](https://github.com/aparrish/gutenberg-poetry-corpus)
- The compressed file is available in your Colab filesystem (e.g. `/content/gutenberg-poetry.ndjson.gz`)

## Approach
1. Load + clean data
2. Build tokenizer (GPT-2 base)
3. Proper train/val split
4. Chunk into blocks (256 context)
5. Fine-tune GPT-2 with proper schedule
6. Monitor overfitting
7. Generate samples

This is how I would structure it in production.

In [ ]:
## 1️⃣ Setup Environment

In [ ]:
# Install dependencies
%pip install -q transformers datasets accelerate bitsandbytes

import torch
import numpy as np
import pandas as pd
import random
import os
import gzip
import json
import requests

from datasets import Dataset
from transformers import (
    GPT2TokenizerFast,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

In [ ]:
## 2️⃣ Load Gutenberg Poetry Corpus

We’ll download the compressed NDJSON file directly from the Gutenberg Poetry Corpus and cache it in the Colab filesystem.

In [ ]:
# Path where we’ll store the Gutenberg Poetry Corpus
corpus_path = "/content/gutenberg-poetry.ndjson.gz"

# Official corpus URL from Allison Parrish’s Gutenberg Poetry Corpus
# See: https://github.com/aparrish/gutenberg-poetry-corpus
corpus_url = "https://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz"

# Download once if not already present
if not os.path.exists(corpus_path):
    print("Downloading Gutenberg Poetry Corpus from:", corpus_url)
    resp = requests.get(corpus_url, stream=True)
    resp.raise_for_status()
    with open(corpus_path, "wb") as f:
        for chunk in resp.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print("Download complete.")
else:
    print("Found existing corpus file at", corpus_path)

# Load lines from the compressed NDJSON corpus
lines = []
with gzip.open(corpus_path, "rt", encoding="utf-8") as f:
    for i, line in enumerate(f):
        obj = json.loads(line)
        s = obj.get("s", "").strip()
        if not s:
            continue
        lines.append(s)

print("Total lines loaded:", len(lines))

# We use `texts` to keep the rest of the pipeline unchanged
texts = lines

In [ ]:
## 3️⃣ Clean Data (Minimal but Important)

We preserve formatting. Poetry depends on line breaks.

In [ ]:
def clean_text(t):
    t = t.strip()
    t = t.replace("\r\n", "\n")
    t = t.replace("\r", "\n")
    return t

texts = [clean_text(t) for t in texts if len(t) > 50]

print("Number of poems:", len(texts))

In [ ]:
## 4️⃣ Combine into Single Corpus

We preserve stanza breaks.

In [ ]:
corpus = "\n\n".join(texts)

print("Total characters:", len(corpus))

# Check token scale later

In [ ]:
## 5️⃣ Initialize Tokenizer

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Add stanza token for stronger structure modeling
tokenizer.add_special_tokens({"additional_special_tokens": ["<STANZA>"]})

# Replace double newlines
corpus = corpus.replace("\n\n", " <STANZA> \n")

In [ ]:
## 6️⃣ Tokenize Corpus

In [ ]:
tokens = tokenizer(corpus, return_tensors="pt")["input_ids"][0]

print("Total tokens:", tokens.shape[0])

# You want 10M+ tokens here.
# If less, consider:
# - concatenating multiple Kaggle poetry datasets
# - adding Gutenberg poetry corpus

In [ ]:
## 7️⃣ Train / Validation Split (Critical)

Never train on 100%.

In [ ]:
split_idx = int(0.95 * len(tokens))
train_tokens = tokens[:split_idx]
val_tokens = tokens[split_idx:]

## 8️⃣ Chunk Into Fixed-Length Blocks

We use block size 256 for poetry.

In [ ]:
block_size = 256

def chunk_tokens(token_tensor):
    examples = []
    for i in range(0, len(token_tensor) - block_size, block_size):
        examples.append(token_tensor[i:i+block_size])
    return examples

train_examples = chunk_tokens(train_tokens)
val_examples = chunk_tokens(val_tokens)

print("Train chunks:", len(train_examples))
print("Val chunks:", len(val_examples))

## 9️⃣ Convert to HuggingFace Dataset

In [ ]:
train_dataset = Dataset.from_dict({"input_ids": train_examples})
val_dataset = Dataset.from_dict({"input_ids": val_examples})

## 🔟 Load GPT-2 Model

In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

## 1️⃣1️⃣ Data Collator

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## 1️⃣2️⃣ Training Arguments (Expert Settings)

These are tuned for stability + minimal overfitting.

**Why these values:**
- `5e-5` → safe for fine-tuning
- `cosine decay` → smoother convergence
- `warmup` prevents early divergence
- `weight_decay` combats overfitting
- `gradient accumulation` for stable large batch
- `load_best_model_at_end` ensures best val checkpoint

In [ ]:
training_args = TrainingArguments(
    output_dir="poetry-gpt2",
    overwrite_output_dir=True,
    
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    
    gradient_accumulation_steps=8,  # effective batch 64
    
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=100,
    
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.03,
    
    lr_scheduler_type="cosine",
    
    fp16=True,
    
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    
    report_to="none"
)

## 1️⃣3️⃣ Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

trainer.train()

## 📉 Monitoring Overfitting

Watch:
- **training loss**
- **validation loss**

If validation loss:
- **decreases then increases** → stop early
- **flat** → LR too low
- **exploding** → LR too high

For 10M tokens, 3–5 epochs is usually enough.

## 🎭 Generate Poetry

In [ ]:
def generate(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    output = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=True,
        temperature=0.9,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate("Moonlight spills across the silent river\n"))

## 🎯 Best Practices Summary

### To prevent overfitting:
- ✅ Proper 95/5 split
- ✅ Cosine decay
- ✅ Weight decay 0.01
- ✅ No more than 5 epochs
- ✅ Load best checkpoint
- ✅ Large effective batch
- ✅ Monitor validation loss
- ✅ Do not crank LR above 1e-4

### To improve poetic quality:
- ✅ Preserve line breaks
- ✅ Add `<STANZA>` token
- ✅ Use temperature 0.8–1.0
- ✅ Adjust top-p for creativity

### If You Want Even Better Stability

Upgrade:
- Use `bitsandbytes` 8-bit Adam
- Use gradient clipping (1.0)
- Use Flash Attention (if GPU supports)

**Now you have:**
- Industrial-grade fine-tuning setup
- Proper data pipeline
- Anti-overfitting strategy
- Stable optimization
- Controlled generation

This is how I would deploy a stylistically consistent GPT-2 fine-tune in production.